## 종만북 chapter 17. 부분 합
N명의 시험점수를 내림차순으로 정렬해 둔 배열 score[]가 있다.  
a등에서 b등까지의 평균 점수를 계산하는 함수 average(a, b)를 만들고 싶다.  
그냥 생각해보면 scores[a]부터 scores[b]까지를 순회하며 각 수를 더하고 이것을 b-a+1로 나누면 된다.  
이러면 반복문의 수행 횟수는 최대 O(N)  
평균점수를 한 번만 계산한다면 이걸로 충분하지만, average()를 여러 번 호출해야 되는 경우엔??  
이런 경우 부분 합(Partial sum)을 이용하여 최적화 할 수 있다.   
  
**부분합이란?** 배열의 각 위치에 대해 배열의 시작부터 현재 위치까지의 원소의 합을 구해 둔 배열.  
socres[]의 부분 합 psum[]의 각 원소를 다음처럼 정의할 수 있다.  
  
$psum[i] = \sum_{j=0}^i scores[j]$  
  
psum을 미리 계산해두면 scores[]의 특정 구간의 합을 O(1)에 구할 수 있다.  
psum[-1] = 0이라고 가정하면, scores[a]부터 scores[b]까지의 합은 $psum[b] - psum[a-1]$이 된다.


#### 부분 합 계산
구간 합을 빠르게 계산하기 위해서 부분 합을 미리 계산해야한다. 부분 합 계산은 매우 쉽다.  
반복문을 통해 구간 합을 구하면 최대 O(N)이므로 구간 합을 두 번 이상 구할 때는 거의 항상 부분 합을 사용하는 것이 이득이다.  

    // 주어진 벡터 a의 부분합을 계산한다.
    vector<int> partialSum(const vector<int>& a) {
        vector<int> ret(a.size());
        ret[0] = a[0];
        for (int i = 1; i < a.size(); i++)
            ret[i] = ret[i - 1] + a[i];
        return ret;
    }
    
    // 어떤 벡터의 부분합 psum[]이 주어질 때, 원래 벡터의 a부터 b까지의 합을 구하기.
    int rangeSum(const vector<int>& psum, int a, int b) {
        if (a == 0) return psum[b];
        return psum[b] - psum[a - 1];
    }
    
매우 간단.  

In [2]:
# 파이썬으로 구현
def partialSum(a):
    n = len(a)
    ret = [0] * n
    ret[0] = a[0]
    for i in range(1, n):
        ret[i] = ret[i - 1] + a[i]
    return ret

a = [1, 4, 8, 2, 9, 23]
print(partialSum(a))

[1, 5, 13, 15, 24, 47]


In [3]:
def rangeSum(psum, a, b):
    if a == 0:
        return psum[b]
    return psum[b] - psum[a - 1]

psum = [1, 5, 13, 15, 24, 47]
print(rangeSum(psum, 2, 5))

42


  
#### 부분 합으로 분산 계산하기
부분 합을 이용하면 합, 평균 외에도 다른 값들을 쉽게 계산할 수 있다. 분산(Variance)를 계산해보자.  
배열 A[]의 구간 A[a, ..., b]의 분산을 다음과 같은 식으로 나타내진다.  
  
$ v = \frac{1}{b-a+1}\cdot\sum_{i=a}^b(A[i] - m_{a, b})^2$     
  
이 식에서 $m_{a, b})^2$는 해당 구간의 평균이다. psum을 이용해 금방 계산 가능하지만 이것만으로는 분산 계산이 어렵다.  
식을 정리해보자.  
  
$ v = \frac{1}{b-a+1}\cdot\sum_{i=a}^b(A[i] - m_{a, b})^2$  
$\  = \frac{1}{b-a+1}\cdot\sum_{i=a}^b(A[i]^2 - 2A[i]\cdot m_{a,b} + {m_{a, b}}^2)$  
$\  = \frac{1}{b-a+1}\cdot\left(\sum_{i=a}^bA[i]^2 - 2m_{a,b}\cdot \sum_{i=a}^b A[i]+(b-a+1){m_{a, b}}^2\right)$  
  
마지막 식의 괄호 안에서 가운데 항과 오른쪽 항은 psum을 이용해 쉽게 계산 할 수 있다.  
$A[i]^2$ 역시 a[]의 각 원소의 제곱의 부분 합을 저장하는 배열을 미리 만들면 O(1)에 계산할 수 있다.  
  
  
    // A[]의 제곱의 부분 합 벡터 sqpsum, A[]의 부분합 벡터 psum이 주어질 때
    // A[a..b]의 분산을 반환한다.
    double variance(const vector<int>& sqpsum, const vector<int>& psum, int a, int b) {
        // 우선 해당 구간의 평균 계산
        double mean = rangeSum(psum, a, b) / double(b - a + 1);
        double ret = rangeSum(sqpsum, a, b) - 2 * mean * rangeSum(psum, a, b) + (b - a + 1) * mean * mean;
        return ret / (b - a + 1);
    }

In [4]:
def variance(sqpsum, psum, a, b):
    mean = rangeSum(psum, a, b) / (b - a + 1)
    ret = rangeSum(sqpsum, a, b) - 2 * mean * rangeSum(psum, a, b) + (b - a + 1) * mean * mean
    return ret / (b - a + 1)

#### 2차원으로 확장
1차원 배열 뿐만 아니라 2차원 배열에서도 부분 합을 사용할 수 있다.  
$A[][]$이 주어질 때, $A[y_1, x_1]$에서 $A[y_2, x_2]$까지의 직사각형 구간의 합을 계산해야 하는 경우,   
$psum[y, x] = \sum_{i=0}^y\sum_{j=0}^xA[i, j]$ 로 주어지는 부분 합 배열을 사용해 구간 합을 빠르게 구할 수 있다.  
  
psum[y, x]는 (0, 0)을 왼쪽 위 칸, (y, x)를 오른쪽 아래 칸으로 갖는 직사각형 구간에 포함된 원소들의 합이다.  
psum을 미리 계산해두면 2차원 배열에서도 우리가 원하는 구간의 합을 쉽게 구할 수 있다.  
$A[y_1, x_1]$에서 $A[y_2, x_2]$직사각형 범위의 합을 $sum(y_1, x_1, y_2, x_2)$라 하면,  
  
$sum(y_1, x_1, y_2, x_2) = psum[y_2, x_2] - psum[y_2, x_1 -1] - psum[y_1 - 1, x_2] + psum[y_1 - 1, x_1 - 1]$이 된다.  

    // 어떤 2차원 배열 A[][]의 부분합 psum[][]이 주어질 때,
    // A[y1, x1]과 A[y2, x2]를 양 끝으로 갖는 부분 배열의 합을 반환한다.
    int gridSum(const vector<vector<int> >& psum, int y1, int x1, int y2, int x2) {
        int ret = psum[y2][x2];
        if (y1 > 0) ret -= psum[y1 - 1][x2];
        if (x1 > 0) ret -= psum[y2][x1 - 1];
        if (y1 > 0 && x1 > 0) ret += psum[y1 - 1][x1 - 1];
        return ret;
    }

In [5]:
def gridSum(psum, x1, y1, x2, y2):
    ret = psum[y2][x2]
    if y1 > 0:
        ret -= psum[y1 - 1][x2]
    if x1 > 0:
        ret -= psum[y2][x1 - 1]
        if y1 > 0:
            ret += psum[y1 - 1][x1 - 1]
    return ret        

#### 예제: 합이 0에 가장 가까운 구간  
양수와 음수가 모두 포함된 배열 A[]가 있을 때, 그 합이 0에 가장 가까운 구간을 찾아보자.
    
    i     0     1    2    3    4    5    6    7    8    9
    -------------------------------------------------------
    A[i]  -14   7    2    3   -8    4   -6    8    9   11
    
합이 0인 구간은 없지만, 0에 가장 가까운 구간은 A[2] ~ A[5]로 그 합이 1이다.  
이 구간을 찾기 위한 방법으로 먼저 A의 모든 구간을 순회하면서 각각의 합을 계산 할 수 있을 것. &rarr; $O(N^2)$  
부분 합을 이용하면 A[i] ~ A[j] 구간의 합은 $\sum_{k=i}^jA[k] = psum[j] - psum[i]$가 된다.  
이 값이 0에 가깝다는 말은 psum[]의 두 값의 차이가 가장 적다는 뜻이다.  
주어진 배열에서 가장 가까운 두 값을 찾기 위해서 배열을 정렬한 뒤 인접한 원소들을 확인할 수 있다.  
정렬에 $O(NlogN)$, 부분 합을 구하는 것과 인접한 원소들을 확인하는 것 모두 O(N)이므로 종합하면 $O(NlogN)$이 된다.  

### 문제: 크리스마스 인형
산타 할아버지가 전세계 어린이 K명에게 인형을 사 주려고 한다.  
인형가게에는 N개의 인형 상자가 한 줄로 진열돼있고, 각 상자 안에 하나 이상의 인형이 있다.  
각 상자에 0부터 N - 1까지 번호를 붙여 놓고 H번 상자부터 T번 상자까지를 주문하여 구입할 수 있다. (H <= T)  
산타할아버지는 주문시마다 상자에 있는 인형들을 모두 꺼내서 K명에게 정확히 같은 수만큼 나누어 주고, 남는 인형이 없도록 한다.  
  
1. 한 번 주문할 수 있다면, 가능한 주문 방법은 몇 가지일까?
2. 여러 번 주문할 수 있다면, 주문이 겹치지 않게 최대 몇 번 주문할 수 있을까?(주문이 겹친다는 말은 한 인형 상자가 두 개 이상의 주문에 중복으로 포함된다는 것)  

#### 입력
첫 줄에 테스트 케이스 개수 C(C<=60)이 주어짐  
각 테스트 케이스 첫 번째 줄에는 인형 상자의 개수 N과 어린이의 수 K가 주어진다. (1 <= N, k <= 100000)  
두 번째 줄에는 0번 인형 상자부터 N-1번 인형 상자까지 각 상 자에 들어있는 인형의 개수 $D_i$가 주어짐.(0<=i<=N-1, 1<=$D_i$<=100000)  
  
#### 출력
1번에 대한 답과 2번에 대한 답을 한 줄에 하나의 빈칸으로 나누어 출력. 1번 답은 20091101로 나눈 나머지를 출력.

#### 풀이
두 가지의 서로 다른 부분 문제 모두 부분 합을 이용해서 풀 수 있다.  
"H에서 T까지 구입 했을 때 남기지 않고 나눠줄 수 있는가?"라는 문제를 부분합으로 답할 수 있다.  
인형의 총 합이 K의 배수여야 하기 때문에 다음과 같은 관계가 성립한다.  
**(psum[T] - psum[H - 1]) mod K = 0**  이것을 전개하면,  
**psum[T] mod K = psum[H - 1] mod K**  
psum에서 중요한 것은 K로 나눈 나머지므로 psum을 $psum[i] = \left(\sum_{j=0}^iD[j]\right) mod K$로 정의하여 푼다.  

#### 구입할 수 있는 방법의 수 계산
1번 문제를 풀어보면, H번째부터 T번째 상자까지의 합을 K명의 어린이에게 모두 나눠줄 수 있다는 말은  
항상 psum[H - 1] = psum[T]라는 뜻이다.  
따라서 psum[]의 각 원소를 모두 같은 것끼리 모아서 이들 중 두 개를 선택하면 한 번의 주문이 된다.   
따라서 숫자 i가 psum에 출현하는 횟수를 $f_i$라고 하면, 우리가 원하는 값은  
$\sum_{j=0}^{K-1}{f_i \choose 2} = \sum_{j=0}^{K - 1}\left(\frac{(f_i(f_i - 1))}{2}\right)$ 이 된다.  
psum[-1] = 0도 빼놓지 않아야 함!!  
  
#### 겹치지 않고 구입할 수 있는 최대 횟수 계산하기
두 번째 문제는 전형적인 동적 계획법 형태. 부분 합을 이용한 동적 계획법으로 풀 수 있다.  
서로 겹치지 않는 부분 구간을 가장 많이 골라내는 함수를 다음과 같이 정의해보자.  
maxBuys(i) = 0번 상자부터 i번 상자까지의 범위 내에서 서로 겹치지 않고 구매할 수 있는 부분 구간의 최대 수  
  
완전 탐색 알고리즘을 만들기 위해 문제의 한 조각만을 해결하고 나머지를 재귀 호출로 해결하자.  
따라서 maxBuys() 함수는 i번 상자를 어떻게 할지 결정하고 나머지를 재귀 호출해서 해결하면 된다.  
i번 상자를 갖고 할 수 있는 일은 두 가지이다.  
1. i번 상자를 사지 않음: 이 상자를 제외하고 나머지 상자들에 대해 재귀호출로 해결. 이 경우의 답은 maxBuys(i - 1)  
2. i번 상자를 삼: 상자는 항상 범위로만 살 수 있기 때문에 j(j<=i)번 상자에서 i번 상자까지 한 번에 사야함. 그러기 위해서 psum[j-1] = psum[i]여야 한다. 만약 j로 선택할 수 있는 상자가 여러 개 있을 수 있다면 가장 뒤에 있는 상자를 구입해야만 이득이다. 구간의 수를 최대화하는 것이 목적이기 때문. 이 경우 답은 maxBuys(j-1) + 1  
  
이때 maxBuys(i)의 답은 두 부분 문제의 답 중 더 큰 쪽.  
  
#### 구현
구현을 더 간단하게 하기 위해 psum[]의 첫 번째 원소 앞에 0을 하나 추가로 집어넣는다. psum[-1]=0이라는 가정을 쉽게 이용하기 위해.  
psum[i] = psum[j-1]인지를 확인할 때 j = 0인 예외를 신경쓰지 않아도 된다. 자주쓰는 기법.  

    // D[]의 부분 합 배열 psum[]과 k가 주어질 때, 몇 가지 방법으로 살 수 있는지 반환한다.
    // psum[]의 첫 번째 워소 전에 0을 삽입했다고 가정한다.
    int waysToBuy(const vector<int>& psum, int k) {
        const int MOD = 20091101;
        int ret = 0;
        // psum[]의 각 값을 몇 번이나 본 적 있는지 기록한다.
        vector<long long> count(k, 0);
        for (int i = 0; i < psum.size(); i++)
            count[psum[i]]++;
        // 두 번 이상 본 적 있다면 이 값 중 두 개를 선택하는 방법의 수를 더한다.
        for (int i = 0; i < k; k++)
            if (count[i] >= 2)
                ret = (ret + ((count[i] * (count[i] - 1)) / 2)) % MOD;
        return ret;
    }
    // D[]의 부분 합 배열 psum[]과 k가 주어질 때, 겹치지 않게 몇 번이나 살 수 있는지 반환.
    int maxBuys(const vecotr<int>& psum, int k) {
        // ret[i] = 첫 번째 상자부터 i번째 상자까지 고려했을 때 살 수 있는 최대 횟수
        vector<int> ret(psum.size(), 0);
        // prev[s] = psum[]이 s였던 마지막 위치
        vector<int> prev(k, -1);
        for (int i = 0; i < psum.size(); i++) {
            // i번째 상자를 고려하지 않는 경우
            if (i > 0)
                ret[i] = ret[i - 1];
            else
                ret[i] = 0;
            // psum[i]를 전에도 본 적이 있으면 prev[psum[i]] + 1부터 여기까지 쭉 사 본다.
            int loc = prev[psum[i]];
            if (loc != -1) ret[i] = max(ret[i], ret[loc] + 1);
            // prev[]에 현재 위치를 기록한다.
            prev[psum[i]] = i;
        }
        return ret.back();
    }
    
wyasToBuy()는 1번 문제의 답을 구하기 위해 psum[]의 각 값의 출현 빈도수를 계산한 후, 각 빈도 f에 대해 ${f \choose 2}$의 합을 계산.   
psum[]의 맨 앞에 0을 넣어두어 예외처리를 하지 않아도 되는 점, 그리고 오버플로우를 피하기 위해 64비트 정수형 long long을 사용했다.  
  
maxBuys()는 완전 탐색 알고리즘을 반복적 동적 계획법으로 바꿔 구현한다.    
이 과정에서 psum의 각 값을 마지막으로 언제 보았는지를 prev[]에 기록하는데,  
이를 이용하여 어느 상자부터 i번 상자까지를 구입해야 하는지 알 수 있다.  
ret[]의 점화식을 다음과 같이 쓸 수 있다.  
  
$ret[i] = \begin{cases} ret[i - 1]\\ ret[prev[psum[i]]] + 1 & \mbox{prev[psum[i]] $\neq$ -1} \end{cases}$  
  
두 합수의 시간 복잡도는 모두 O(N+K)  

#### 더 공부할 거리
구간 트리(Segment Tree)는 구간 합의 확장에 가까운 개념으로, 구간 합 외에도 구간 곱, 구간의 최소값/최대값, 최대 출현 빈도 등을 빠르게 풀 수 있게 해준다.